# Build a Tool-Calling Agentic AI Research Assistant with LangChain

This demo will cover building AI Agents with the legacy LangChain `AgentExecutor`. These are fine for getting started, but for working with more advanced agents and having more finer control, LangChain recommends to use LangGraph, which we cover in other courses.

Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to stop.

![](https://i.imgur.com/1uVnBAm.png)



## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install markitdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 9.0 MB/s eta 0:00:00


## Enter Open AI API Key

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [4]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

Enter Tavily Search API Key: ··········


## Enter WeatherAPI API Key

Get a free API key from [here](https://www.weatherapi.com/signup.aspx)

In [5]:
WEATHER_API_KEY = getpass('Enter WeatherAPI API Key: ')

Enter WeatherAPI API Key: ··········


## Setup Environment Variables

In [6]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Create Tools

Here we create two custom tools which are wrappers on top of the [Tavily API](https://tavily.com/#api) and [WeatherAPI](https://www.weatherapi.com/)

- Web Search tool with information extraction
- Weather tool

![](https://i.imgur.com/TyPAYXE.png)

In [7]:
from langchain_core.tools import tool
from markitdown import MarkItDown
from langchain_community.tools.tavily_search import TavilySearchResults
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, TimeoutError
import requests
import json
from warnings import filterwarnings
filterwarnings('ignore')

tavily_tool = TavilySearchResults(max_results=5,
                                  search_depth='advanced',
                                  include_answer=False,
                                  include_raw_content=True)
# certain websites won't let you crawl them unless you specify a user-agent
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br"
})
md = MarkItDown(requests_session=session)

@tool
def search_web_extract_info(query: str) -> list:
    """Search the web for a query and extracts useful information from the search links."""
    print('Calling web search tool')
    results = tavily_tool.invoke(query)
    docs = []

    def extract_content(url):
        """Helper function to extract content from a URL."""
        extracted_info = md.convert(url)
        text_title = extracted_info.title.strip()
        text_content = extracted_info.text_content.strip()
        return text_title + '\n' + text_content
    # parallelize execution of different urls
    with ThreadPoolExecutor() as executor:
        for result in tqdm(results):
            try:
                future = executor.submit(extract_content, result['url'])
                # Wait for up to 15 seconds for the task to complete
                content = future.result(timeout=15)
                docs.append(content)
            except TimeoutError:
                print(f"Extraction timed out for url: {result['url']}")
            except Exception as e:
                print(f"Error extracting from url: {result['url']} - {e}")

    return docs


@tool
def get_weather(query: str) -> list:
    """Search weatherapi to get the current weather of the queried location."""
    print('Calling weather tool')
    base_url = "http://api.weatherapi.com/v1/current.json"
    complete_url = f"{base_url}?key={WEATHER_API_KEY}&q={query}"

    response = requests.get(complete_url)
    data = response.json()
    if data.get("location"):
        return data
    else:
        return "Weather Data Not Found"

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


## Test Tool Calling with LLM

In [8]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]

chatgpt_with_tools = chatgpt.bind_tools(tools)

In [9]:
prompt = "Get details of Microsoft's earnings call Q4 2024"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'search_web_extract_info',
  'args': {'query': 'Microsoft earnings call Q4 2024 details'},
  'id': 'call_JbExenn37WbQ4P5K3PznrYzw',
  'type': 'tool_call'}]

In [10]:
prompt = "how is the weather in Bangalore today"
response = chatgpt_with_tools.invoke(prompt)
response.tool_calls

[{'name': 'get_weather',
  'args': {'query': 'Bangalore'},
  'id': 'call_shdm3EoSFkZTbx0ky5W4wMeK',
  'type': 'tool_call'}]

## Build and Test AI Agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent to bind the tools to the agent with a prompt. We will also add in the capability to store historical conversations as memory

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYS_PROMPT = """Act as a helpful assistant.
                You run in a loop of Thought, Action, PAUSE, Observation.
                At the end of the loop, you output an Answer.
                Use Thought to describe your thoughts about the question you have been asked.
                Use Action to run one of the actions available to you - then return PAUSE.
                Observation will be the result of running those actions.
                Repeat till you get to the answer for the given user query.

                Use the following workflow format:
                  Question: the input task you must solve
                  Thought: you should always think about what to do
                  Action: the action to take which can be any of the following:
                            - break it into smaller steps if needed
                            - see if you can answer the given task with your trained knowledge
                            - call the most relevant tools at your disposal mentioned below in case you need more information
                  Action Input: the input to the action
                  Observation: the result of the action
                  ... (this Thought/Action/Action Input/Observation can repeat N times)
                  Thought: I now know the final answer
                  Final Answer: the final answer to the original input question

                Tools at your disposal to perform tasks as needed:
                  - get_weather: whenever user asks get the weather of a place.
                  - search_web_extract_info: whenever user asks for specific information or if you don't know the answer.
             """

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{query}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

prompt_template.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Act as a helpful assistant.\n                You run in a loop of Thought, Action, PAUSE, Observation.\n                At the end of the loop, you output an Answer.\n                Use Thought to describe your thoughts about the question you have been asked.\n                Use Action to run one of the actions available to you - then return PAUSE.\n                Observation will be the result of running those actions.\n                Repeat till you get to the answer for the given user query.\n\n                Use the following workflow format:\n                  Question: the input task you must solve\n                  Thought: you should always think about what to do\n                  Action: the action to take which can be any of the following:\n                            - break it into smaller steps if needed\n                            - see if you can

Now, we can initalize the agent with the LLM, the prompt, and the tools.

The agent is responsible for taking in input and deciding what actions to take.

REMEMBER the Agent does not execute those actions - that is done by the AgentExecutor

Note that we are passing in the model `chatgpt`, not `chatgpt_with_tools`.

That is because `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

This should ideally be used with an LLM which supports tool \ function calling

In [15]:
from langchain.agents import create_tool_calling_agent

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [search_web_extract_info, get_weather]
agent = create_tool_calling_agent(chatgpt, tools, prompt_template)

Finally, we combine the `agent` (the brains) with the `tools` inside the `AgentExecutor` (which will repeatedly call the agent and execute tools).

In [16]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               early_stopping_method='force',
                               max_iterations=10)

In [18]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = chatgpt.invoke(query)
print(response.content)

I'm sorry, but I don't have access to real-time data or the ability to retrieve specific documents such as Nvidia's Q4 2024 earnings call. However, I can help you understand what typically happens during an earnings call and what key points are usually discussed. 

During an earnings call, a company typically covers:

1. **Financial Performance**: Discussion of revenue, profit margins, net income, and earnings per share compared to previous quarters and analyst expectations.

2. **Business Segments**: Performance of different business units or segments, such as gaming, data center, professional visualization, and automotive for Nvidia.

3. **Market Trends**: Insights into market conditions and trends affecting the business, such as demand for GPUs, AI advancements, or supply chain issues.

4. **Guidance**: Forward-looking statements about expected financial performance, including revenue and earnings forecasts for the next quarter or fiscal year.

5. **Strategic Initiatives**: Updates 

In [19]:
query = """Summarize the key points discussed in Nvidia's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})

Calling web search tool


 80%|████████  | 4/5 [00:05<00:01,  1.49s/it]

Error extracting from url: https://www.investiment.io/symbols/NVDA/earnings/transcripts/2024/4 - 'NoneType' object has no attribute 'strip'


100%|██████████| 5/5 [00:05<00:00,  1.19s/it]

Error extracting from url: https://investor.nvidia.com/news/press-release-details/2024/NVIDIA-Announces-Financial-Results-for-Fourth-Quarter-and-Fiscal-2024/ - 403 Client Error: Forbidden for url: https://investor.nvidia.com/news/press-release-details/2024/NVIDIA-Announces-Financial-Results-for-Fourth-Quarter-and-Fiscal-2024/


In [20]:
response

{'query': "Summarize the key points discussed in Nvidia's Q4 2024 earnings call",
 'output': "Nvidia's Q4 2024 earnings call highlighted several key points:\n\n1. **Record Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, a 22% increase from the previous quarter and a 265% increase year-over-year. The full-year revenue reached $60.9 billion, up 126% from the previous year.\n\n2. **Data Center Growth**: The data center segment achieved a record $18.4 billion in revenue for the quarter, marking a 27% rise from Q3 and a 409% year-over-year growth. This growth was driven by the adoption of Nvidia's Hopper GPU computing platform and InfiniBand networking.\n\n3. **AI and Accelerated Computing**: Nvidia emphasized the transition from general-purpose to accelerated computing, driven by the demand for AI and generative AI applications. The company is focusing on building AI factories to refine data and produce intelligence.\n\n4. **Gross Margin and Profitabil

In [21]:
from IPython.display import display, Markdown

display(Markdown(response['output']))

Nvidia's Q4 2024 earnings call highlighted several key points:

1. **Record Financial Performance**: Nvidia reported a record quarterly revenue of $22.1 billion, a 22% increase from the previous quarter and a 265% increase year-over-year. The full-year revenue reached $60.9 billion, up 126% from the previous year.

2. **Data Center Growth**: The data center segment achieved a record $18.4 billion in revenue for the quarter, marking a 27% rise from Q3 and a 409% year-over-year growth. This growth was driven by the adoption of Nvidia's Hopper GPU computing platform and InfiniBand networking.

3. **AI and Accelerated Computing**: Nvidia emphasized the transition from general-purpose to accelerated computing, driven by the demand for AI and generative AI applications. The company is focusing on building AI factories to refine data and produce intelligence.

4. **Gross Margin and Profitability**: The gross margin improved to 76.0% in Q4, up from 74.0% in Q3 and 63.3% the previous year. Net income for Q4 was $12.3 billion, a 33% increase from Q3 and a 769% increase year-over-year.

5. **Gaming and Other Segments**: Gaming revenue was $2.87 billion, up 56% year-over-year. Nvidia also reported growth in professional visualization and automotive segments.

6. **Future Outlook**: Nvidia expects Q1 2025 revenue to be around $24 billion, with continued growth in data center and professional visualization segments. The company plans to invest in new product cycles and innovations to maintain its market position.

Overall, Nvidia's strong financial performance was driven by its leadership in AI and accelerated computing, with significant contributions from its data center and gaming segments.

In [22]:
query = """Summarize the key points discussed in Intel's Q4 2024 earnings call"""
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling web search tool


100%|██████████| 5/5 [00:05<00:00,  1.02s/it]


Intel's Q4 2024 earnings call highlighted several key points:

1. **Financial Performance**: Intel reported a profit of $2.7 billion for Q4 2024, compared to a loss of $0.7 billion in the same period the previous year. The earnings per share (EPS) were $0.63, up from a loss of $0.16 per share last year. Adjusted earnings were $2.3 billion or $0.54 per share, surpassing analysts' expectations of $0.45 per share.

2. **Revenue**: The company generated $15.4 billion in revenue for the quarter, an increase from $14.0 billion in the previous year.

3. **Guidance**: For the next quarter, Intel provided guidance for EPS at $0.13 and revenue between $12.2 billion and $13.2 billion.

These results indicate a strong recovery and performance improvement for Intel, exceeding market expectations and setting a positive outlook for the upcoming quarter.

In [23]:
query = """which company's future outlook looks to be better?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

To provide an accurate assessment of a company's future outlook, I would need to know which specific companies you are interested in comparing. Please provide the names of the companies you would like to evaluate.

In [24]:
query = """how is the weather in Bangalore today?
           show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling weather tool


The weather in Bangalore today is as follows:

- **Temperature**: 16.7°C (62.1°F)
- **Condition**: Partly Cloudy
- **Wind**: 10.8 kph (6.7 mph) from the East
- **Pressure**: 1013.0 mb (29.91 in)
- **Precipitation**: 0.0 mm
- **Humidity**: 92%
- **Cloud Cover**: 28%
- **Feels Like**: 16.7°C (62.1°F)
- **Dew Point**: 15.4°C (59.7°F)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 0.0
- **Wind Gusts**: 19.0 kph (11.8 mph)

The data was last updated at 6:30 AM local time.

In [25]:
query = """how is the weather in Dubai today?
           show detailed statistics
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Calling weather tool


The weather in Dubai today is clear. Here are the detailed statistics:

- **Temperature**: 17.6°C (63.7°F)
- **Condition**: Clear
- **Wind**: 12.3 mph (19.8 kph) from the East (93°)
- **Pressure**: 1021.0 mb (30.16 in)
- **Precipitation**: 0.0 mm (0.0 in)
- **Humidity**: 43%
- **Cloud Cover**: 0%
- **Feels Like**: 17.6°C (63.7°F)
- **Dew Point**: 5.0°C (41.1°F)
- **Visibility**: 10.0 km (6.0 miles)
- **UV Index**: 0.0
- **Wind Gusts**: 21.9 mph (35.2 kph)

The current time in Dubai is 5:04 AM.

In [26]:
query = """which city is hotter?
        """
response = agent_executor.invoke({"query": query})
display(Markdown(response['output']))

Question: The user is asking which city is hotter, but they haven't specified the cities they are interested in. I need more information to proceed.

Thought: I need to ask the user for the names of the cities they want to compare in terms of temperature.

Final Answer: Could you please specify the names of the cities you would like to compare to determine which one is hotter?

The agent is doing pretty well but unfortunately it doesn't remember conversations. We will use some user-session based memory to store this and dive deeper into this in the next video.